In [1]:
! ls ../../data/

CrowdFlowerAnnotations.txt  Flickr8k.token.txt	       machine_translation
ExpertAnnotations.txt	    Flickr_8k.devImages.txt    readme.txt
Flicker8k_smaller	    Flickr_8k.testImages.txt
Flickr8k.lemma.token.txt    Flickr_8k.trainImages.txt


In [2]:
! pip3 install jieba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 2.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314477 sha256=ed27d19a84f32bfb6048db6e791987820e2c52697ae582f47ab1202f2dc61d03
  Stored in directory: /root/.cache/pip/wheels/24/aa/17/5bc7c72e9a37990a9620cc3aad0acad1564dcff6dbc2359de3
Successfully built jieba

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GRU, Dense, Layer, Embedding
import numpy as np
import random
import jieba

batch_size = 16  
epochs = 15  
latent_dim = 128  # Latent dimensionality of the encoding space.
num_samples = 2000  # Number of samples to train on.
data_path = '../../data/machine_translation/cmn.txt'

input_texts = []
target_texts = []
input_tokens = set()
target_tokens = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')

# for line in lines[: min(num_samples, len(lines) - 1)]:
for line in random.sample(lines, num_samples):
    input_text, target_text = line.split('\t')
    
    # We use "\t" as the "start sequence" and "\n" as "end sequence"
    target_text = '\t' + target_text + '\n'
    
    tmp = []
    for token in input_text.split(" "):
        token = token.replace(",", "").replace(".", "").replace("?", "").lower()
        tmp.append(token)
        if token not in input_tokens:
            input_tokens.add(token)
    input_texts.append(tmp)
    
    tmp = []
    for token in jieba.cut(target_text, cut_all=False):
        tmp.append(token)
        if token not in target_tokens:
            target_tokens.add(token)
    target_texts.append(tmp)

input_tokens = list(input_tokens)
target_tokens = list(target_tokens)
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

print("\ninput data set:")
print(input_texts[:10])
print("\ntarget data set:")
print(target_texts[:10])

2025-06-28 01:19:45.947404: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-06-28 01:19:45.947558: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 2.061 seconds.
Prefix dict has been built successfully.


Number of samples: 2000
Number of unique input tokens: 2184
Number of unique output tokens: 3261
Max sequence length for inputs: 21
Max sequence length for outputs: 23

input data set:
[['he', 'knows', 'new', 'york', 'inside', 'out'], ['you', "aren't", 'kids', 'anymore'], ['if', "it's", 'at', 'all', 'possible', "i'd", 'like', 'you', 'to', 'take', 'part', 'in', 'the', 'next', 'meeting'], ['the', 'sky', 'is', 'full', 'of', 'stars'], ['she', 'made', 'cookies', 'for', 'the', 'children'], ['this', 'factory', 'manufactures', 'cd', 'players'], ['tom', 'was', 'a', 'nurse'], ['i', 'lost', 'my', 'money'], ['i', 'can', 'work', 'with', 'anyone'], ['we', 'are', 'looking', 'for', 'a', 'nice', 'house', 'to', 'live', 'in']]

target data set:
[['\t', '他', '对', '纽约', '了如指掌', '。', '\n'], ['\t', '你們', '不再', '是', '小孩', '了', '。', '\n'], ['\t', '如果', '可能', '的', '話', ',', ' ', '我', '希望', '你', '參加', '下', '一次', '的', '會議', '。', '\n'], ['\t', '天空', '布满', '星辰', '。', '\n'], ['\t', '她', '為', '孩子', '們', '做', '了', '餅乾

In [2]:
input_token_index = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_token_index = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

print("\ninput index:")
print(input_token_index)

print("\ntarget index:")
print(target_token_index)


input index:
{"weather's": 0, 'impressed': 1, 'solve': 2, 'hearing': 3, 'members': 4, 'preparing': 5, 'experience': 6, "wasn't": 7, 'succeeded': 8, 'visit': 9, 'project': 10, 'remain': 11, 'denmark': 12, 'managed': 13, 'or': 14, 'answer': 15, 'scientist': 16, 'faint': 17, 'sat': 18, 'awake': 19, '1989': 20, 'severely': 21, 'support': 22, 'simple': 23, 'dollars': 24, 'tends': 25, 'pen': 26, 'spoon': 27, 'car': 28, 'text': 29, "can't": 30, 'system': 31, 'liked': 32, 'week': 33, 'grew': 34, 'thing': 35, 'early': 36, 'computer': 37, 'recovered': 38, 'genius': 39, 'stressed': 40, 'yourself': 41, 'players': 42, 'instead': 43, 'send': 44, 'classroom': 45, 'stole': 46, 'comes': 47, 'crashed': 48, 'sick': 49, 'dress': 50, 'shine': 51, 'policeman': 52, 'gave': 53, '20': 54, 'objection': 55, 'vowed': 56, 'scotland': 57, 'dinner': 58, 'pay': 59, 'studying': 60, 'under': 61, 'lady': 62, 'then': 63, 'laughed': 64, 'best': 65, 'returned': 66, 'speaks': 67, 'arrived': 68, 'boyfriend': 69, 'strongest'

In [3]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, token in enumerate(input_text):
        encoder_input_data[i, t] = input_token_index[token]
    for t, token in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t] = target_token_index[token]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[token]] = 1.

print("The shape is (batch_size, time_steps, input_dim)")
print("so it will be (number of x, length of each x, dimension of x)")
print("\nencoder input shape:")
print(encoder_input_data.shape)
print("\ndecoder input shape:")
print(decoder_input_data.shape)
print("\ndecoder target shape:")
print(decoder_target_data.shape)
            
print("\nencoder input data (1-hot):")
print(encoder_input_data[-2:])
print("\ndecoder input data (1-hot):")
print(decoder_input_data[-2:])
print("\ndecoder target data (1-hot):")
print(decoder_target_data[-2:])

The shape is (batch_size, time_steps, input_dim)
so it will be (number of x, length of each x, dimension of x)

encoder input shape:
(2000, 21)

decoder input shape:
(2000, 23)

decoder target shape:
(2000, 23, 3261)

encoder input data (1-hot):
[[ 981. 1468.  982. 1911. 1946.  195.    0.    0.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [ 981. 1703. 1831.  915. 1516. 1987. 1355. 1159. 1909. 1617.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.]]

decoder input data (1-hot):
[[ 311. 1950. 1573.  845. 2707.  213. 2550.  767.    0.    0.    0.    0.
     0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]
 [ 311. 1616. 1315. 2804.   42. 1297. 2232. 2474.   21. 2577. 1776. 2550.
   767.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.]]

decoder target data (1-hot):
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 

In [4]:
class Encoder(Model):
    def __init__(self, vocab_size, latent_dim, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.latent_dim = latent_dim
        self.lstm = GRU(self.latent_dim,
                        return_sequences=True,
                        return_state=True,
                        recurrent_initializer='glorot_uniform')
        self.embedding = Embedding(vocab_size, latent_dim, mask_zero=True)

    def call(self, x):
        print(f"x: {x.shape}")
        x = self.embedding(x)
        output, state = self.lstm(x)
        return output, state


encoder_inputs = Input(shape=(None, ))
encoder = Encoder(num_encoder_tokens, latent_dim, batch_size)
encoder_outputs, state = encoder(encoder_inputs)

print(f"encoder_inputs: {encoder_inputs.shape}")
print(f"state: {state.shape}")
print(f"encoder_outputs: {encoder_outputs.shape}")

2025-06-28 01:22:05.409258: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2025-06-28 01:22:05.409780: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-06-28 01:22:05.409830: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2025-06-28 01:22:05.409952: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (2780c4aad032): /proc/driver/nvidia/version does not exist
2025-06-28 01:22:05.412775: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the ap

x: (None, None)
encoder_inputs: (None, None)
state: (None, 128)
encoder_outputs: (None, None, 128)


In [6]:
import tensorflow as tf

class CrossAttention(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super().__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(key_dim=units, num_heads=1, **kwargs)
        self.layernorm = tf.keras.layers.LayerNormalization()
        self.add = tf.keras.layers.Add()

    def call(self, x, context):
        print(f'x: {x.shape}')
        print(f'context: {context.shape}')

        attn_output, attn_scores = self.mha(
            query=x,
            value=context,
            return_attention_scores=True)

        print(f'x: {x.shape}')
        print(f'score: {attn_scores.shape}')

        attn_scores = tf.reduce_mean(attn_scores, axis=1)
        print(f'score: {attn_scores.shape}')
        
        # NOTE: I will return it, not save it to a variable
        # self.last_attention_weights = attn_scores

        x = self.add([x, attn_output])
        x = self.layernorm(x)

        return x, attn_scores

# We can also try not to use hidden in Decoder (i.e., remove `initial_state=hidden`), all information comes from attention layers

In [7]:
class Decoder(Model):
    def __init__(self, vocab_size, latent_dim, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.latent_dim = latent_dim
        self.lstm = GRU(latent_dim,
                        return_sequences=True,
                        return_state=True,
                        recurrent_initializer='glorot_uniform')
        self.embedding = Embedding(vocab_size, latent_dim, mask_zero=True)
        self.fc = Dense(vocab_size, activation='softmax')
        self.attention = CrossAttention(self.latent_dim)

    def call(self, x, hidden, enc_output):
        print(f"x: {x.shape}")
        print(f"hidden: {hidden.shape}")
        print(f"enc_output: {enc_output.shape}")
        
        x = self.embedding(x)
        rnn_output, rnn_state = self.lstm(x, initial_state=hidden)
        print(f"rnn_output: {rnn_output.shape}")
        
        res, attention_weights = self.attention(rnn_output, enc_output)
        y = self.fc(res)

        return y, rnn_state


decoder_inputs = Input(shape=(None, ))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder = Decoder(num_decoder_tokens, latent_dim, batch_size)
decoder_outputs, _ = decoder(decoder_inputs, hidden=state, enc_output=encoder_outputs)


print(f"decoder_inputs: {decoder_inputs.shape}")
print(f"state: {state.shape}")
print(f"decoder_outputs: {decoder_outputs.shape}")

x: (None, None)
hidden: (None, 128)
enc_output: (None, None, 128)
rnn_output: (None, None, 128)
x: (None, None, 128)
context: (None, None, 128)
x: (None, None, 128)
score: (None, 1, None, None)
score: (None, None, None)
decoder_inputs: (None, None)
state: (None, 128)
decoder_outputs: (None, None, 3261)


In [8]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
encoder (Encoder)               ((None, None, 128),  378624      input_1[0][0]                    
__________________________________________________________________________________________________
decoder (Decoder)               ((None, None, 3261), 1003453     input_2[0][0]                    
                                                                 encoder[0][0]                

In [9]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
        batch_size=batch_size,
        epochs=2,
        validation_split=0.2)

2025-06-28 01:28:54.036676: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2025-06-28 01:28:54.074908: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2196495000 Hz


Epoch 1/2
x: (16, 21)
x: (16, 23)
hidden: (16, 128)
enc_output: (16, 21, 128)
rnn_output: (16, 23, 128)
x: (16, 23, 128)
context: (16, 21, 128)
x: (16, 23, 128)
score: (16, 1, 23, 21)
score: (16, 23, 21)
x: (16, 21)
x: (16, 23)
hidden: (16, 128)
enc_output: (16, 21, 128)
rnn_output: (16, 23, 128)
x: (16, 23, 128)
context: (16, 21, 128)
x: (16, 23, 128)
score: (16, 1, 23, 21)
score: (16, 23, 21)
100/100 [==============================] - ETA: 0s - loss: 2.0204x: (16, 21)
x: (16, 23)
hidden: (16, 128)
enc_output: (16, 21, 128)
rnn_output: (16, 23, 128)
x: (16, 23, 128)
context: (16, 21, 128)
x: (16, 23, 128)
score: (16, 1, 23, 21)
score: (16, 23, 21)
100/100 [==============================] - 26s 158ms/step - loss: 2.0188 - val_loss: 1.6474
Epoch 2/2
100/100 [==============================] - 10s 101ms/step - loss: 1.5988 - val_loss: 1.6143


In [10]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [11]:
encoder_model = Model(encoder_inputs, [encoder_outputs, state])
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
encoder (Encoder)            ((None, None, 128), (None 378624    
Total params: 378,624
Trainable params: 378,624
Non-trainable params: 0
_________________________________________________________________


In [12]:
decoder_state_input = Input(shape=(latent_dim,))
the_encoder_output = Input(shape=(None, latent_dim,))

decoder_outputs, decoder_state = decoder(
    decoder_inputs, hidden=decoder_state_input, enc_output=the_encoder_output)

decoder_model = Model(
    [decoder_inputs] + [decoder_state_input] + [the_encoder_output],
    [decoder_outputs] + [decoder_state])

decoder_model.summary()

x: (None, None)
hidden: (None, 128)
enc_output: (None, None, 128)
rnn_output: (None, None, 128)
x: (None, None, 128)
context: (None, None, 128)
x: (None, None, 128)
score: (None, 1, None, None)
score: (None, None, None)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 128)]  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 128)]        0                                            
______________________________________________________________________

# There are 2 ways to do decoding

In [13]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    encoder_output_value, states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['\t']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        print(f"target_seq: {target_seq}")
        
        output_tokens, the_decoder_state = decoder_model.predict(
            [target_seq] + [states_value] + [encoder_output_value])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        # target_seq = np.zeros((1, 1))
        # target_seq[0, 0] = sampled_token_index
        
        #NOTE: maybe another way to do it???? maybe not
        target_seq = np.append(target_seq, [[sampled_token_index]], axis=1)

        # Update states
        states_value = the_decoder_state

    return decoded_sentence


for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

x: (None, 21)
target_seq: [[311.]]
x: (None, 1)
hidden: (None, 128)
enc_output: (None, 21, 128)
rnn_output: (None, 1, 128)
x: (None, 1, 128)
context: (None, 21, 128)
x: (None, 1, 128)
score: (None, 1, 1, 21)
score: (None, 1, 21)
target_seq: [[311. 394.]]
x: (None, 2)
hidden: (None, 128)
enc_output: (None, 21, 128)
rnn_output: (None, 2, 128)
x: (None, 2, 128)
context: (None, 21, 128)
x: (None, 2, 128)
score: (None, 1, 2, 21)
score: (None, 2, 21)
target_seq: [[ 311.  394. 1155.]]
x: (None, None)
hidden: (None, 128)
enc_output: (None, 21, 128)
rnn_output: (None, None, 128)
x: (None, None, 128)
context: (None, 21, 128)
x: (None, None, 128)
score: (None, 1, None, 21)
score: (None, None, 21)
target_seq: [[ 311.  394. 1155. 1449.]]
target_seq: [[ 311.  394. 1155. 1449. 2550.]]
-
Input sentence: ['he', 'knows', 'new', 'york', 'inside', 'out']
Decoded sentence: 他是的。

target_seq: [[311.]]
target_seq: [[ 311. 2589.]]
target_seq: [[ 311. 2589. 1449.]]
target_seq: [[ 311. 2589. 1449. 2550.]]
-
Inpu